In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [5]:
df = pd.read_csv('data.csv',delimiter=';')

In [6]:
# Bağımlı değişken
target = 'default'

# Bağımsız değişkenler
categorical_columns = ['status_last_archived_0_24m', 
                       'status_2nd_last_archived_0_24m', 
                       'status_3rd_last_archived_0_24m', 
                       'status_max_archived_0_6_months', 
                       'status_max_archived_0_12_months', 
                       'status_max_archived_0_24_months', 
                       'name_in_email', 
                       'worst_status_active_inv', 
                       'merchant_group', 'merchant_category', 
                       'account_worst_status_6_12m', 
                       'account_worst_status_3_6m', 
                       'account_worst_status_12_24m', 
                       'account_worst_status_0_3m', 
                       'account_status']

In [7]:
finding_data = df[df['default'].isnull()]

In [8]:
new_df = df[df['default'].notnull()]
new_df['default'].value_counts()

default
0.0    88688
1.0     1288
Name: count, dtype: int64

In [9]:
X = new_df.drop(target, axis=1)
X = X.drop('uuid', axis=1)
y = new_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
finding_data_cleaned = finding_data.drop(target, axis=1)
finding_data_cleaned = finding_data_cleaned.drop('uuid', axis=1)

In [11]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encode the categorical columns
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_encoded = pd.DataFrame(encoder.fit_transform(X_train[categorical_columns]))
X_test_encoded = pd.DataFrame(encoder.transform(X_test[categorical_columns]))
finding_dataset_encoded = pd.DataFrame(encoder.transform(finding_data_cleaned[categorical_columns]))


/opt/homebrew/Caskroom/miniforge/base/envs/work/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
from sklearn.ensemble import RandomForestClassifier

# Sınıflandırıcıyı oluşturun
classifier = RandomForestClassifier()

# Sınıflandırıcıyı eğitin
classifier.fit(X_train_encoded, y_train)

RandomForestClassifier()

In [13]:
# Tahmin yapın
y_pred = classifier.predict(X_test_encoded)

# Performansı değerlendirin
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9848299622138253


In [14]:
y_finding_data_pred = classifier.predict(finding_dataset_encoded)
y_finding_data_pred_df = pd.DataFrame(y_finding_data_pred, columns=['default'])

finding_data.reset_index(drop=True, inplace=True)
finding_data['default'] = y_finding_data_pred_df['default']

/var/folders/gj/f88mgj1d6qx3zfj1q56fmdrr0000gn/T/ipykernel_9661/250697581.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finding_data['default'] = y_finding_data_pred_df['default']


In [15]:
finding_data.to_csv('finding_data.csv', index=False, sep=';')

In [16]:
# save classifier
import pickle
pickle.dump(classifier, open('classifier.pkl', 'wb'))

# save encoder
pickle.dump(encoder, open('encoder.pkl', 'wb'))